# Building Machine Learning Classifiers: Model selection

### Read in & clean text

In [5]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import pickle
from sklearn.externals import joblib
import random

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

In [6]:
pd.set_option('max_colwidth', 800)
data.head()

,label,body_text,body_len,punct%
0,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,128,4.7
1,ham,"Nah I don't think he goes to usf, he lives around here though",49,4.1
2,ham,Even my brother is not like to speak with me. They treat me like aids patent.,62,3.2
3,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,28,7.1
4,ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune,135,4.4


In [7]:
data.shape

(5567, 4)

### Split into train/test

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data[['body_text', 'body_len', 'punct%']],\
                                                    data['label'], test_size=0.2)


### Vectorize text

In [9]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
tfidf_vect_fit = tfidf_vect.fit(X_train['body_text'])

tfidf_train = tfidf_vect_fit.transform(X_train['body_text'])
tfidf_test = tfidf_vect_fit.transform(X_test['body_text'])

X_train_vect = pd.concat([X_train[['body_len', 'punct%']].reset_index(drop=True), 
           pd.DataFrame(tfidf_train.toarray())], axis=1)
X_test_vect = pd.concat([X_test[['body_len', 'punct%']].reset_index(drop=True), 
           pd.DataFrame(tfidf_test.toarray())], axis=1)

X_train_vect.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,7153,7154,7155,7156,7157,7158,7159,7160,7161,7162
0,27,3.7,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,57,1.8,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,92,14.1,0.212125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,51,2.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,35,2.9,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
X_train_vect.shape

(4453, 7165)

### Final evaluation of models

In [11]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import time

In [12]:
rf = RandomForestClassifier(n_estimators=150, max_depth=None, n_jobs=-1)

rf_model = rf.fit(X_train_vect, y_train)
y_pred = rf_model.predict(X_test_vect)

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
print('Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Precision: 1.0 / Recall: 0.778 / Accuracy: 0.971


In [13]:
gb = GradientBoostingClassifier(n_estimators=150, max_depth=11)

gb_model = gb.fit(X_train_vect, y_train)
y_pred = gb_model.predict(X_test_vect)

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
print('Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Precision: 0.914 / Recall: 0.812 / Accuracy: 0.966


In [14]:
# Save Random Forest model
with open('sms_spam_classifier_RF.pkl','wb') as file:
    pickle.dump(rf,file)

In [15]:
# Save model vocabulary of vectorizer
pickle.dump(tfidf_vect.vocabulary_,open("features.pkl","wb"))

In [16]:
joblib.dump(tfidf_vect_fit, 'tfidf.pkl') 

['tfidf.pkl']

**Predicting with Model**

In [134]:
k = random.randint(0,data.shape[0])
sms = data['body_text'][k]
sms

'Can U get 2 phone NOW? I wanna chat 2 set up meet Call me NOW on 09096102316 U can cum here 2moro Luv JANE xx Calls£1/minmoremobsEMSPOBox45PO139WA'

In [135]:
data['label'][k]

'spam'

In [136]:
predict_data = [sms]
df = pd.DataFrame(predict_data,columns=['body_text'])
df['body_len'] = df['body_text'].apply(lambda x: len(x) - x.count(" "))
df['punct%'] = df['body_text'].apply(lambda x: count_punct(x))

In [137]:
df

,body_text,body_len,punct%
0,Can U get 2 phone NOW? I wanna chat 2 set up meet Call me NOW on 09096102316 U can cum here 2moro Luv JANE xx Calls£1/minmoremobsEMSPOBox45PO139WA,120,1.7


In [138]:
tfidf_predict = tfidf_vect_fit.transform(df['body_text'])
X_predict_vect = pd.concat([df[['body_len', 'punct%']].reset_index(drop=True), 
           pd.DataFrame(tfidf_predict.toarray())], axis=1)

In [139]:
X_predict_vect

,body_len,punct%,0,1,2,3,4,5,6,7,...,7153,7154,7155,7156,7157,7158,7159,7160,7161,7162
0,120,1.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [140]:
gb_model.predict(X_predict_vect)

array(['spam'], dtype=object)

**Predicting with saved model**

In [141]:
predict_data2 = [sms]
df2 = pd.DataFrame(predict_data2,columns=['body_text'])
df2['body_len'] = df2['body_text'].apply(lambda x: len(x) - x.count(" "))
df2['punct%'] = df2['body_text'].apply(lambda x: count_punct(x))

In [142]:
df2

,body_text,body_len,punct%
0,Can U get 2 phone NOW? I wanna chat 2 set up meet Call me NOW on 09096102316 U can cum here 2moro Luv JANE xx Calls£1/minmoremobsEMSPOBox45PO139WA,120,1.7


In [143]:
tfidf_n = joblib.load('tfidf.pkl') 

In [144]:
tfidf_predict_sm = tfidf_n.transform(df['body_text'])
X_predict_vect_sm = pd.concat([df[['body_len', 'punct%']].reset_index(drop=True), 
           pd.DataFrame(tfidf_predict.toarray())], axis=1)

In [145]:
X_predict_vect_sm

,body_len,punct%,0,1,2,3,4,5,6,7,...,7153,7154,7155,7156,7157,7158,7159,7160,7161,7162
0,120,1.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [146]:
# Load classifier
with open('sms_spam_classifier_RF.pkl', 'rb') as f:
    model = pickle.load(f)

In [147]:
model.predict(X_predict_vect_sm)

array(['spam'], dtype=object)